In [1]:
import os
import re
import pandas as pd
from config import *
from time import sleep
from selenium import webdriver
from multiprocessing import Process
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options


In [2]:
try:
    current_path = os.path.dirname(os.path.abspath(__file__))
except:
    current_path = '.'

In [3]:
def init_driver(gecko_driver='', user_agent='', load_images=True, is_headless=False):
    '''
        This function is just to set up some of default for browser
    '''
    firefox_profile = webdriver.FirefoxProfile()
    
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)
    firefox_profile.set_preference("media.volume_scale", "0.0")
    firefox_profile.set_preference("dom.webnotifications.enabled", False)
    if user_agent != '':
        firefox_profile.set_preference("general.useragent.override", user_agent)
    if not load_images:
        firefox_profile.set_preference('permissions.default.image', 2)

    options = Options()
    options.headless = is_headless
    
    driver = webdriver.Firefox(options=options,
                               executable_path=f'{current_path}/{gecko_driver}',
                               firefox_profile=firefox_profile)
    
    return driver

In [4]:
def get_url(url, driver):
    '''
    Argument:
        url of any page to get
        driver that was inilized
    return:
        True
    '''
    driver.get(url)
#     driver.refresh()
#     sleep(2)
    return True


In [5]:
def drug_content(contents, commercialnames):
    all_data = ""
    commercialnames = []
    if commercialnames:
        for text_in in contents:
            commercialnames.append(text_in.text)
        return commercialnames
    else:
        for text_in in contents:
            all_data += text_in.text +' '
        return all_data

In [6]:
def one_drug_details(driver3):
    one_drug_data = {}
    selector = driver3.find_elements_by_css_selector
    try:
        commercialnames       = drug_content(selector('#commercialnames div tbody tr td'), True)
        commercialnames       = re.findall('[^A-Za-z 0-9().\n]+', commercialnames)
        one_drug_data = {
#             'ar_name':      selector(),
#             'en_name':      drug_content(selector('#whatisdrug div p'), False),
            'description':      drug_content(selector('#whatisdrug div p'), False),
            'method':           drug_content(selector('#method div'), False),
            'frequency':        drug_content(selector('#frequency div'), False),
            'dosagerange':      drug_content(selector('#dosagerange div'), False),
            'affectiontime':    drug_content(selector('#affectiontime div'), False),
            'affectionperiod':  drug_content(selector('#affectionperiod div'), False),
            'nutrition':        drug_content(selector('#nutrition div'), False),
            'storage':          drug_content(selector('#storage div'), False),
            'forgotenmeal':     drug_content(selector('#forgotenmeal div'), False),
            'medecinestop':     drug_content(selector('#medecinestop div'), False),
            'excessdosing':     drug_content(selector('#excessdosing div'), False),
            'pregnancy':        drug_content(selector('#pregnancy div'), False),
            'breastfeed':       drug_content(selector('#breastfeed div'), False),
            'children':         drug_content(selector('#children div'), False),
            'olds':             drug_content(selector('#olds div'), False),
            'driving':          drug_content(selector('#driving div'), False),
            'surgery':          drug_content(selector('#surgery div'), False),
            'sideeffects':      drug_content(selector('#sideeffects div'), False),
            'commercialnames':      commercialnames,
        }
    except Exception as e:
# send exception to log folder
        file = open("logs_files/WebTeb_Scraper.log","+a")
        file.write("This error related to function one_drug_details of WebTeb Scraper file\n" 
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return one_drug_data

In [7]:
def all_drugs_of_one_alphabeta(driver2, char_index):
    all_drugs = driver2.find_elements_by_css_selector('tbody tr .text-right')
    dicts_drugs_alph = {}
    ar_name              =[]
    en_name              =[]
    description          =[]
    method               =[]
    frequency            =[]
    dosagerange          =[]
    affectiontime        =[]
    affectionperiod      =[]
    nutrition            =[]
    storage              =[]
    forgotenmeal         =[]
    medecinestop         =[]
    excessdosing         =[]
    pregnancy            =[]
    breastfeed           =[]
    children             =[]
    olds                 =[]
    driving              =[]
    surgery              =[]
    sideeffects          =[]
    commercialnames      =[]
    for drug in all_drugs:
        drug_url = drug.find_element_by_css_selector('a').get_attribute('href')
        driver3 = init_driver(gecko_driver,user_agent=user_agent)
        get_url(drug_url, driver3)
        ar = driver3.find_element_by_css_selector('header h1').text
        en = driver3.find_element_by_css_selector('header .en-name').text 
        one_drug_data = one_drug_details(driver3)
        ar_name.append(ar)
        en_name.append(en)
        description.append(one_drug_data['description'])
        method.append(one_drug_data['method'])
        frequency.append(one_drug_data['frequency'])
        dosagerange.append(one_drug_data['dosagerange'])
        affectiontime.append(one_drug_data['affectiontime'])
        affectionperiod.append(one_drug_data['affectionperiod'])
        nutrition.append(one_drug_data['nutrition'])
        storage.append(one_drug_data['storage'])
        forgotenmeal.append(one_drug_data['forgotenmeal'])
        medecinestop.append(one_drug_data['medecinestop'])
        excessdosing.append(one_drug_data['excessdosing'])
        pregnancy.append(one_drug_data['pregnancy'])
        breastfeed.append(one_drug_data['breastfeed'])
        children.append(one_drug_data['children'])
        olds.append(one_drug_data['olds'])
        driving.append(one_drug_data['driving'])
        surgery.append(one_drug_data['surgery'])
        sideeffects.append(one_drug_data['sideeffects'])
        commercialnames.append(one_drug_data['commercialnames'])
        driver3.quit()

    dicts_drugs_alph = {
        'ar_name':               ar_name,
        'en_name':               en_name,
        'description':           description,
        'method':                method,
        'frequency':             frequency,
        'dosagerange':           dosagerange,
        'affectiontime':         affectiontime,
        'affectionperiod':       affectionperiod,
        'nutrition':             nutrition,
        'storage':               storage,
        'forgotenmeal':          forgotenmeal,
        'medecinestop':          medecinestop,
        'excessdosing':          excessdosing,
        'pregnancy':             pregnancy,
        'breastfeed':            breastfeed,
        'children':              children,
        'olds':                  olds,
        'driving':               driving,
        'surgery':               surgery,
        'sideeffects':           sideeffects,
        'commercialnames':       commercialnames,
    }
    df = pd.DataFrame(dicts_drugs_alph)
    df.to_csv('data/char_'+ str(char_index) +'.csv', index=False)
    df.to_excel('data/char_'+ str(char_index) + 'char.xlsx', index=False)
    return True

In [9]:
def all_alphabeta(alphabetas):
    char_index = 1
    for alphabeta in alphabetas:
        if char_index > 6:
            alphabeta_url = alphabeta.find_element_by_css_selector('a').get_attribute('href')
            driver2 = init_driver(gecko_driver,user_agent=user_agent)
            get_url(alphabeta_url, driver2)
            _ = all_drugs_of_one_alphabeta(driver2, char_index)
            driver2.quit()
            print("Now We in Char number: " + str(char_index))
        char_index +=1
    return True


In [ ]:
driver = init_driver(gecko_driver,user_agent=user_agent)
get_url(web_teb, driver)
show_more = driver.find_elements_by_css_selector('section .acc-head .bold')
show_more = show_more[2]
show_more.click()
alphabetas = driver.find_elements_by_css_selector('.alphabeta.ar li')
all_alphabeta(alphabetas)

### Call the main function with multithreading process

In [ ]:
# if __name__ == '__main__':
#     p1 = Process(target=scrap_pages, args=(jumia_url_mobile,1))
#     p1.start()
#     p2 = Process(target=scrap_pages, args=(jumia_url_electronics,1))
#     p2.start()
#     p3 = Process(target=scrap_pages, args=(jumia_url_computing,1))
#     p3.start()
#     p1.join()
#     p2.join()
#     p3.join()